<a href="https://colab.research.google.com/github/jingwen0/systemeAQ/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
from numpy import unicode
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from gensim.models import KeyedVectors, Word2Vec
from sklearn import model_selection,preprocessing
import re
import unicodedata
import csv
import pandas as pd
import numpy as np
import textblob
import string
import time
import json
import pprint
from itertools import chain
import nltk
from nltk import word_tokenize

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [23]:
def seg_sentence(data):
    # seg_words = word_tokenize(data, language="french")
    seg_words = data.split(" ")# 空格分词

    return seg_words

In [24]:
def read_csv(filename):
    # 加载数据
    df = pd.read_csv(filename, sep=',',error_bad_lines=False)
    print(df.head())
    print(df.shape)
    print(list(df.columns))
    text = df['text']
    label = df['label']
    print("总条数：",len(label),'\t',len(text))
    return text, label

In [25]:
!mkdir data
!wget -P data https://raw.githubusercontent.com/weekend1111/sa/main/corpus_annote.csv

mkdir: cannot create directory ‘data’: File exists
--2022-06-26 10:38:46--  https://raw.githubusercontent.com/weekend1111/sa/main/corpus_annote.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746317 (729K) [text/plain]
Saving to: ‘data/corpus_annote.csv.1’

corpus_annote.csv.1 100%[===================>] 728.83K  --.-KB/s    in 0.04s   

2022-06-26 10:38:46 (15.9 MB/s) - ‘data/corpus_annote.csv.1’ saved [746317/746317]



In [26]:
def cleaning_text(text):
    #Input: list of sentences
    #Output: list of sentences
    output = [re.sub('  ', ' ', x.lower()) for x in text]
    return output

In [27]:
# 获取原始数据
filename = './data/corpus_annote.csv'
texts, labels = read_csv(filename)

# 数据预处理工作
clean_texts = cleaning_text(list(texts))#分词

     id                                               text  label
0  7101  Mon trésor, mon trésor ! On va se faire bouffe...      1
1  6114  Julian, ici toi père. Même si en ce moment, ba...      0
2   438  Depuis la prison, il est encore plus méfiant e...     -1
3  1904  J'ai fait ma part. Je veux l'argent. Pas ce soir.     -1
4  2299  Non. Eh bien, croyez-moi, je sais ce que je fais.      1
(9988, 3)
['id', 'text', 'label']
总条数： 9988 	 9988


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
def data_process(old_texts,texts, labels):
    seg_sentences = []
    new_labels = []
    sentences = []
    for i in range(len(labels)):
        data = texts[i]
        label = int(labels[i])
        old_data = old_texts[i]
        if label == 1 or  label == -1 or label == 0:

            word_tokens = seg_sentence(data)# 分词
            
            sentences.append(data)
            seg_sentences.append(word_tokens)
            new_labels.append(label+1)
        else:
            continue
    return seg_sentences,sentences,new_labels

In [29]:
seg_sentences,sentences,new_labels = data_process(texts, clean_texts, labels)
print(new_labels[0],'\t',sentences[0],'\t',seg_sentences[0])

2 	 mon trésor, mon trésor ! on va se faire bouffer les couilles ! 	 ['mon', 'trésor,', 'mon', 'trésor', '!', 'on', 'va', 'se', 'faire', 'bouffer', 'les', 'couilles', '!']


In [30]:
# 切分数据
ids = [i for i in range(len(new_labels))]
train_x_id, valid_x_id, train_y, valid_y = model_selection.train_test_split(ids, new_labels, test_size=0.1, random_state=5) #random_state为空时说明每次随机切分都不一样。测试时要固定测试集，随便设置一个数值。最好的办法还是一开始就把训练集和测试集分别写不同文件，永久固定
sentences = np.array(sentences)
train_x = sentences[np.array(train_x_id)]# 8989
valid_x = sentences[np.array(valid_x_id)]# 999

seg_sentences = np.array(seg_sentences)
train_seg_x = seg_sentences[np.array(train_x_id)]
valid_seg_x = seg_sentences[np.array(valid_x_id)]
print(len(train_x))
print(len(valid_x))

8989
999


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [31]:
def encode_samples(tokenized_samples, word_to_idx,unk_id):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(unk_id)
        features.append(feature)
    return features

In [32]:
def pad_samples(features, maxlen=500, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features

In [33]:
def update_vector(weight, wvmodel, word_to_idx):
    # wv_size = len(wvmodel)
    # print('wv_size:',wv_size)

    # wv_words = wvmodel.key_to_index
    wv_words = wvmodel.index2word 
    
    unk_count = 0
    
    for word,index in word_to_idx.items():

        if word in wv_words:
        
            vector = np.array(wvmodel.word_vec(word))
            weight[index, :] = torch.from_numpy(vector)
            #print('++++++++',word, index)
        else:
            #print("~~~~UNK:", word, index) 
            vector = weight[1]#UNK
            weight[index, :] = vector
            unk_count+=1
    print('unk_count:',unk_count)# UNK1888

    return weight

In [34]:
class LSTMNet(nn.Module):
    # 初始化，准备工作
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, weight, labels, use_gpu, **kwargs):
        super(LSTMNet, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.use_gpu = use_gpu
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0)
        if self.bidirectional:
            self.decoder = nn.Linear(num_hiddens * 4, labels)
        else:
            self.decoder = nn.Linear(num_hiddens * 2, labels)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([1, 0, 2]))
        encoding = torch.cat([states[0], states[-1]], dim=1)
        outputs = self.decoder(encoding)
        return outputs

In [35]:
import os
device = torch.device("cuda:0")
def train(vocab_size, wvmodel, word_to_idx, train_features, train_labels, test_features, test_labels):

    # 设置超参数
    num_epochs = 20 # 数据跑几次
    embed_size = wvmodel.vector_size # 200维
    print('embed_size:',embed_size)
    num_hiddens = embed_size
    num_layers = 2
    bidirectional = True
    batch_size = 16 # 一次两个句子
    labels = 3
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"# Pytorch中使用指定的GPU
    #lr = 0.8
    lr = 5e-4 # 0.001学习率
    gpu_id = 0
    

    use_gpu = False
    
    # 使用词向量来初始化词表的表征为，全量词袋
    weight = torch.rand(vocab_size+2, embed_size)
    # print(weight[2])# 检查是否更新
    weight = update_vector(weight, wvmodel, word_to_idx)
    #print(weight[2])
    print('weight.shape:',weight.shape)
    
    # 初始化模型，可替换
    net = LSTMNet(vocab_size=vocab_size+2, embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional, weight=weight,
                   labels=labels, use_gpu=use_gpu)
    print("device: ", device)            
    net.to(device)
    
    # exit()
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr)
    
    # 加载数据，并随机打乱训练集
    train_set = torch.utils.data.TensorDataset(train_features, train_labels)
    test_set = torch.utils.data.TensorDataset(test_features, test_labels)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False)
    print("test_iter", test_iter)
    
    # 开始训练
    for epoch in range(num_epochs):
        start = time.time()
        train_loss, test_losses = 0, 0
        train_acc, test_acc = 0, 0
        n, m = 0, 0
        for feature, label in train_iter:
            n += 1
            net.train()
            net.zero_grad()
            feature=feature.to(device)#
            label=label.to(device)
            score = net(feature)
            loss = loss_function(score, label)
            loss.backward()
            optimizer.step()
            train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                     dim=1), label.cpu())
            train_loss += loss
            #print("train_loss",train_loss)
        with torch.no_grad():
            for test_feature, test_label in test_iter:
                m += 1
                net.eval()
                test_feature=test_feature.to(device)
                test_label=test_label.to(device)

                test_score = net(test_feature)
                test_loss = loss_function(test_score, test_label)
                test_acc += accuracy_score(torch.argmax(test_score.cpu().data,
                                                        dim=1), test_label.cpu())
                test_losses += test_loss
                #print("test_losses",test_losses)
        end = time.time()
        runtime = end - start
        print('epoch: %d, train loss: %.4f, train acc: %.2f, test loss: %.4f, test acc: %.2f, time: %.2f' %
              (epoch, train_loss.data / n, train_acc / n, test_losses.data / m, test_acc / m, runtime))


In [36]:
!wget -P data https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin

--2022-06-26 10:38:47--  https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin
Resolving embeddings.net (embeddings.net)... 212.107.17.115
Connecting to embeddings.net (embeddings.net)|212.107.17.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126052447 (120M) [application/octet-stream]
Saving to: ‘data/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin.1’

frWac_non_lem_no_po   8%[>                   ]   9.77M  1.65MB/s    eta 67s    ^C


In [37]:
from gensim.models import KeyedVectors, Word2Vec

In [38]:
# 加载词向量
model = KeyedVectors.load_word2vec_format("data/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin", binary=True)

In [39]:
# 获取训练集的词表
vocab = set(chain(*train_seg_x))
vocab_size = len(vocab)
#print(vocab)
print('vocab_size:',vocab_size)#训练集词表太小了,测试集UNK会很多

vocab_size: 17152


In [40]:
# 构建查表字典 id:word, word:id   pad = 0 unk = 1
pad_id = 0
unk_id = 1
word_to_idx = {word: i+2 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = unk_id
idx_to_word = {i+2: word for i, word in enumerate(vocab)}
idx_to_word[unk_id] = '<unk>'

In [41]:
# 句子转换成id，并补齐长度
max_lenth = 20
train_ids = encode_samples(train_seg_x, word_to_idx, unk_id) # unk为1
train_features = torch.tensor(pad_samples(train_ids, max_lenth, pad_id))# pad为0
train_labels = torch.tensor(train_y)
    
valid_ids = encode_samples(valid_seg_x, word_to_idx, unk_id) # unk为1
valid_features = torch.tensor(pad_samples(valid_ids, max_lenth, pad_id))# pad为0
valid_labels = torch.tensor(valid_y)
print(train_ids[0])
print(train_features[0])
print(train_features.shape) #[9000, 50]
print(valid_features.shape) #[1000, 50]

[12137, 11397, 9544, 3296, 8385, 5475, 3624, 8262, 17080, 6824, 8963, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tensor([12137, 11397,  9544,  3296,  8385,  5475,  3624,  8262, 17080,  6824,
         8963,     0,     0,     0,     0,     0,     0,     0,     0,     0])
torch.Size([8989, 20])
torch.Size([999, 20])


In [42]:
# 训练
train(vocab_size, model, word_to_idx, train_features, train_labels, valid_features, valid_labels)

embed_size: 200
unk_count: 9346
weight.shape: torch.Size([17154, 200])
device:  cuda:0
test_iter <torch.utils.data.dataloader.DataLoader object at 0x7f3f4b33dcd0>
开始训练。。。
epoch: 0, train loss: 1.0883, train acc: 0.41, test loss: 1.0792, test acc: 0.48, time: 6.34
开始训练。。。
epoch: 1, train loss: 1.0721, train acc: 0.47, test loss: 1.0666, test acc: 0.48, time: 6.12
开始训练。。。
epoch: 2, train loss: 1.0627, train acc: 0.47, test loss: 1.0592, test acc: 0.48, time: 6.16
开始训练。。。
epoch: 3, train loss: 1.0573, train acc: 0.47, test loss: 1.0550, test acc: 0.48, time: 6.14
开始训练。。。
epoch: 4, train loss: 1.0541, train acc: 0.47, test loss: 1.0526, test acc: 0.48, time: 6.15
开始训练。。。
epoch: 5, train loss: 1.0523, train acc: 0.47, test loss: 1.0512, test acc: 0.48, time: 6.15
开始训练。。。
epoch: 6, train loss: 1.0513, train acc: 0.47, test loss: 1.0504, test acc: 0.48, time: 6.16
开始训练。。。
epoch: 7, train loss: 1.0508, train acc: 0.47, test loss: 1.0500, test acc: 0.48, time: 6.12
开始训练。。。
epoch: 8, train loss: